In [ ]:
import numpy as np
from numpy.random import randint, rand
from collections import defaultdict

class TimetableGA:
    def __init__(self, n_bits, n_pop, r_cross, r_mut, n_classes, n_rooms, max_generations=1000, tolerance=0.0001):
        self.n_bits = n_bits
        self.n_pop = n_pop
        self.r_cross = r_cross
        self.r_mut = r_mut
        self.n_classes = n_classes
        self.n_rooms = n_rooms
        self.population = [self.random_bitstring() for _ in range(n_pop)]
        self.best_solution = None
        self.best_score = float('inf')
        self.max_generations = max_generations
        self.tolerance = tolerance
        self.penalties = {
            'professor_courses': 15,
            'section_courses': 15,
            'adjacent_days': 12,
            'room_capacity': 15,
            'double_booking': 12
        }

    def random_bitstring(self):
        return randint(0, 2, self.n_bits).tolist()

    def decode(self, bitstring):
        timetable = []
        step = 13
        for i in range(0, len(bitstring), step):
            chromosome = bitstring[i:i+step]
            timetable.append({
                'course': chromosome[0],
                'theory_lab': chromosome[1],
                'section': chromosome[2],
                'section_strength': chromosome[3],
                'professor': chromosome[4],
                'first_lecture_day': chromosome[5],
                'first_lecture_timeslot': chromosome[6],
                'first_lecture_room': chromosome[7],
                'first_lecture_room_size': chromosome[8],
                'second_lecture_day': chromosome[9],
                'second_lecture_timeslot': chromosome[10],
                'second_lecture_room': chromosome[11],
                'second_lecture_room_size': chromosome[12]
            })
        return timetable

def calculate_conflicts(self, timetable):
    conflicts = 0
    professor_courses = defaultdict(int)
    section_courses = defaultdict(int)
    room_usage = defaultdict(list)
    class_rooms = defaultdict(set)  # Track rooms used by each class
    floor_changes = defaultdict(list)  # Track floor changes for teachers/students

    for cls in timetable:
        professor_courses[cls['professor']] += 1
        section_courses[cls['section']] += 1

        room_key1 = (cls['first_lecture_day'], cls['first_lecture_timeslot'], cls['first_lecture_room'])
        room_key2 = (cls['second_lecture_day'], cls['second_lecture_timeslot'], cls['second_lecture_room'])

        room_usage[room_key1].append(cls)
        room_usage[room_key2].append(cls)

        # Track rooms and floors used by each class
        class_rooms[cls['course']].add(cls['first_lecture_room'])
        class_rooms[cls['course']].add(cls['second_lecture_room'])

        floor_changes[cls['professor']].append(cls['first_lecture_room'])
        floor_changes[cls['professor']].append(cls['second_lecture_room'])
        floor_changes[cls['section']].append(cls['first_lecture_room'])
        floor_changes[cls['section']].append(cls['second_lecture_room'])

    for cls in timetable:
        # Hard Constraints
        if professor_courses[cls['professor']] > 3:
            conflicts += self.penalties['professor_courses']
        if section_courses[cls['section']] > 5:
            conflicts += self.penalties['section_courses']
        if abs(cls['first_lecture_day'] - cls['second_lecture_day']) <= 1:
            conflicts += self.penalties['adjacent_days']
        if cls['section_strength'] > cls['first_lecture_room_size']:
            conflicts += self.penalties['room_capacity']
        if cls['section_strength'] > cls['second_lecture_room_size']:
            conflicts += self.penalties['room_capacity']

        # Soft Constraints
        # Theory classes in morning, labs in afternoon
        if cls['theory_lab'] == 0 and cls['first_lecture_timeslot'] > 2:
            conflicts += 5  # Penalty for theory class not in morning
        if cls['theory_lab'] == 1 and cls['first_lecture_timeslot'] <= 2:
            conflicts += 5  # Penalty for lab class not in afternoon

        # Classes in same room
        if len(class_rooms[cls['course']]) > 1:
            conflicts += 3  # Penalty for class not in same room

    # Minimize floor traversal
    for floors in floor_changes.values():
        if len(set(floors)) > 1:
            conflicts += len(set(floors)) - 1  # Penalty for floor changes

    # Room double booking
    for room, classes in room_usage.items():
        if len(classes) > 1:
            conflicts += self.penalties['double_booking'] * (len(classes) - 1)

    return conflicts

    def fitness_function(self, bitstring):
        timetable = self.decode(bitstring)
        conflicts = self.calculate_conflicts(timetable)
        return 1.0 / (1.0 + conflicts)

    def selection(self, scores, k=3):
        selection_ix = randint(len(scores))
        for ix in randint(0, len(scores), k-1):
            if scores[ix] < scores[selection_ix]:
                selection_ix = ix
        return self.population[selection_ix]

    def crossover(self, p1, p2):
        c1, c2 = p1.copy(), p2.copy()
        if rand() < self.r_cross:
            pt = randint(1, len(p1)-2)
            c1 = p1[:pt] + p2[pt:]
            c2 = p2[:pt] + p1[pt:]
        return [c1, c2]

    def mutation(self, bitstring, gen):
        adaptive_r_mut = self.r_mut / (1 + gen / 100)
        for i in range(len(bitstring)):
            if rand() < adaptive_r_mut:
                bitstring[i] = 1 - bitstring[i]

    def repair_timetable(self, timetable):
        # Adjust lecture days and timeslots
        for cls in timetable:
            if cls['first_lecture_day'] == cls['second_lecture_day']:
                cls['second_lecture_day'] = (cls['first_lecture_day'] + 2) % 5
            if cls['first_lecture_room'] == cls['second_lecture_room'] and cls['first_lecture_timeslot'] == cls['second_lecture_timeslot']:
                cls['second_lecture_timeslot'] = (cls['first_lecture_timeslot'] + 1) % 5

        # Resolve room double bookings
        room_usage = defaultdict(list)
        for cls in timetable:
            room_key1 = (cls['first_lecture_day'], cls['first_lecture_timeslot'], cls['first_lecture_room'])
            room_key2 = (cls['second_lecture_day'], cls['second_lecture_timeslot'], cls['second_lecture_room'])
            room_usage[room_key1].append(cls)
            room_usage[room_key2].append(cls)

        for room, classes in room_usage.items():
            if len(classes) > 1:
                for cls in classes[1:]:
                    cls['first_lecture_timeslot'] = (cls['first_lecture_timeslot'] + 1) % 5
                    cls['second_lecture_timeslot'] = (cls['second_lecture_timeslot'] + 1) % 5

        return timetable

    def dynamic_penalty_adjustment(self):
        # Increase penalties during optimization
        for key in self.penalties:
            self.penalties[key] *= 1.5

    def run(self):
        initial_conflicts = self.calculate_conflicts(self.decode(self.population[0]))
        print(f"Initial number of conflicts: {initial_conflicts}")

        gen = 0

        gen = 0
        while gen < self.max_generations and self.best_score > self.tolerance:
            scores = [self.fitness_function(c) for c in self.population]
            for i in range(self.n_pop):
                if scores[i] < self.best_score:
                    self.best_solution, self.best_score = self.population[i], scores[i]
            selected = [self.selection(scores) for _ in range(self.n_pop)]

            # Select elite individuals
            elite_size = max(1, self.n_pop // 10)
            elites = sorted(zip(scores, self.population), key=lambda x: x[0])[:elite_size]
            children = [elite[1] for elite in elites]

            # Perform crossover and mutation
            while len(children) < self.n_pop:
                p1, p2 = selected[randint(0, len(selected))], selected[randint(0, len(selected))]
                for c in self.crossover(p1, p2):
                    self.mutation(c, gen)
                    c = self.decode(c)
                    c = self.repair_timetable(c)
                    c = [bit for chrom in c for bit in chrom.values()]
                    children.append(c)

            self.population = children[:self.n_pop]
            if gen % 5 == 0:
                self.dynamic_penalty_adjustment()

            gen += 1

        return self.best_solution, self.best_score


def decode_timetable(bitstring):
    step = 13
    timetable = []
    for i in range(0, len(bitstring), step):
        chromosome = bitstring[i:i+step]
        timetable.append({
            'course': chromosome[0],
            'theory_lab': chromosome[1],
            'section': chromosome[2],
            'section_strength': chromosome[3],
            'professor': chromosome[4],
            'first_lecture_day': chromosome[5],
            'first_lecture_timeslot': chromosome[6],
            'first_lecture_room': chromosome[7],
            'first_lecture_room_size': chromosome[8],
            'second_lecture_day': chromosome[9],
            'second_lecture_timeslot': chromosome[10],
            'second_lecture_room': chromosome[11],
            'second_lecture_room_size': chromosome[12]
        })
    return timetable

def validate_timetable(timetable):
    professor_courses = defaultdict(int)
    section_courses = defaultdict(int)
    room_usage = defaultdict(list)
    conflicts = 0

    for cls in timetable:
        professor_courses[cls['professor']] += 1
        section_courses[cls['section']] += 1

        room_key1 = (cls['first_lecture_day'], cls['first_lecture_timeslot'], cls['first_lecture_room'])
        room_key2 = (cls['second_lecture_day'], cls['second_lecture_timeslot'], cls['second_lecture_room'])

        room_usage[room_key1].append(cls)
        room_usage[room_key2].append(cls)

    for cls in timetable:
        if professor_courses[cls['professor']] > 3:
            conflicts += 1
            print(f"Conflict: Professor {cls['professor']} has more than 3 courses.")
        if section_courses[cls['section']] > 5:
            conflicts += 1
            print(f"Conflict: Section {cls['section']} has more than 5 courses.")
        if abs(cls['first_lecture_day'] - cls['second_lecture_day']) <= 1:
            conflicts += 1
            print(f"Conflict: Adjacent or same days for course {cls['course']} in section {cls['section']}.")
        if cls['section_strength'] > cls['first_lecture_room_size']:
            conflicts += 1
            print(f"Conflict: Room capacity exceeded for the first lecture of course {cls['course']}.")
        if cls['section_strength'] > cls['second_lecture_room_size']:
            conflicts += 1
            print(f"Conflict: Room capacity exceeded for the second lecture of course {cls['course']}.")

    for room, classes in room_usage.items():
        if len(classes) > 1:
            conflicts += len(classes) - 1
            print(f"Conflict: Room {room[2]} is double-booked on day {room[0]} during slot {room[1]}.")

    return conflicts

def print_timetable(timetable):
    for cls in timetable:
        print(f"Course: {cls['course']}, Theory/Lab: {cls['theory_lab']}, Section: {cls['section']}, "
              f"Strength: {cls['section_strength']}, Professor: {cls['professor']}, "
              f"1st Day: {cls['first_lecture_day']}, 1st Slot: {cls['first_lecture_timeslot']}, "
              f"1st Room: {cls['first_lecture_room']}, 1st Room Size: {cls['first_lecture_room_size']}, "
              f"2nd Day: {cls['second_lecture_day']}, 2nd Slot: {cls['second_lecture_timeslot']}, "
              f"2nd Room: {cls['second_lecture_room']}, 2nd Room Size: {cls['second_lecture_room_size']}")

# Parameters
n_bits = 130
n_pop = 100
r_cross = 0.85
r_mut = 0.06  # Further increased mutation rate
n_classes = 10
n_rooms = 5

# Run the genetic algorithm
ga = TimetableGA(n_bits, n_pop, r_cross, r_mut, n_classes, n_rooms)
best_solution, best_score = ga.run()

print("Optimization completed.")
print(f"Best solution: {best_solution}")
print(f"Best fitness score: {best_score:.5f}")

# Decode and validate the best timetable
decoded_timetable = decode_timetable(best_solution)
print_timetable(decoded_timetable)
print("\nValidation of Timetable:")
total_conflicts = validate_timetable(decoded_timetable)
print(f"Total Conflicts: {total_conflicts}")


Initial number of conflicts: 606
Optimization completed.
Best solution: [1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1]
Best fitness score: 0.00008
Course: 1, Theory/Lab: 1, Section: 1, Strength: 1, Professor: 0, 1st Day: 1, 1st Slot: 1, 1st Room: 0, 1st Room Size: 0, 2nd Day: 0, 2nd Slot: 0, 2nd Room: 0, 2nd Room Size: 0
Course: 0, Theory/Lab: 1, Section: 0, Strength: 1, Professor: 1, 1st Day: 1, 1st Slot: 1, 1st Room: 1, 1st Room Size: 0, 2nd Day: 0, 2nd Slot: 0, 2nd Room: 0, 2nd Room Size: 0
Course: 1, Theory/Lab: 1, Section: 1, Strength: 1, Professor: 0, 1st Day: 0, 1st Slot: 0, 1st Room: 1, 1st Room Size: 0, 2nd Day: 0, 2nd Slot: 0, 2nd Room: 0